In [1]:
import tensorflow as tf

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os
from os import listdir

from tensorflow.keras.layers import Input, Dense, Flatten, Conv2DTranspose, MaxPooling2D, Dropout, BatchNormalization, LeakyReLU, Reshape, Conv2D
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [11]:
# importing the file and check the directories
from PIL import Image
import os, numpy as np
folder = '/Users/t.kan/Desktop/CNN/test_images'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

ims = [read(os.path.join(folder, filename)) for filename in os.listdir(folder)]
im_array = np.array(ims, dtype='uint8')
im_array = (im_array / 255.0)*2 - 1

In [13]:
latent_dim = 100

In [21]:
def build_generator(letent_dim):
    model = Sequential()
    
    model.add(Dense(256*8*8,input_shape=(latent_dim,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((8,8,256)))
    
    model.add(Conv2DTranspose(128, (5,5), strides = 4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, (5,5), strides = 4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(128, (5,5), strides = 4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(3, (3,3), padding = "same", activation = "tanh"))
    
    i = Input(shape=(latent_dim,))
    x = model(i)
    
    return Model(i,x)

In [22]:
generator_check = build_generator(latent_dim)
generator_check.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_3 (Sequential)   (None, 512, 512, 3)       3297027   
                                                                 
Total params: 3,297,027
Trainable params: 3,297,027
Non-trainable params: 0
_________________________________________________________________


In [25]:
def build_discriminator(image_size):
    model = Sequential()
    
    model.add(Conv2D(64,(5,5), strides=4, padding = "same", input_shape=image_size))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128,(5,5), strides=4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128,(5,5), strides=4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(256,(5,5), strides=4, padding = "same"))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    
    i = Input(shape=image_size)
    x = model(i)
    
    return Model(i,x)

In [26]:
discriminator_check = build_discriminator((512,512,3))
discriminator_check.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 sequential_5 (Sequential)   (None, 1)                 1440001   
                                                                 
Total params: 1,440,001
Trainable params: 1,440,001
Non-trainable params: 0
_________________________________________________________________


In [27]:
discriminator = build_discriminator([512,512,3])
discriminator.compile(optimizer = Adam(0.0002, 0.5),
                      loss = "binary_crossentropy",
                      metrics = ["accuracy"])

generator = build_generator(latent_dim)

In [28]:
z = Input(shape=(latent_dim,))

img = generator(z)

discriminator.trainable = False

fake_pred = discriminator(img)

combined_model = Model(z, fake_pred)

combined_model.compile(optimizer = Adam(0.0002, 0.5),
                       loss = "binary_crossentropy")

combined_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100)]             0         
                                                                 
 model_3 (Functional)        (None, 512, 512, 3)       3297027   
                                                                 
 model_2 (Functional)        (None, 1)                 1440001   
                                                                 
Total params: 4,737,028
Trainable params: 3,297,027
Non-trainable params: 1,440,001
_________________________________________________________________


In [33]:
batch_size = 8
epochs = 10001
sample_period = 1000

ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

d_losses = []
g_losses = []

if not os.path.exists("/Users/t.kan/Desktop/CNN/gan_images"):
    os.makedirs("/Users/t.kan/Desktop/CNN/gan_images")
    
if not os.path.exists("/Users/t.kan/Desktop/CNN/model_dir"):
    os.makedirs("/Users/t.kan/Desktop/CNN/model_dir")

In [31]:
def sample_images(epoch):
    rows, cols = 5, 5
    noise = np.random.randn(rows*cols, latent_dim)
    imgs = generator.predict(noise)

  # rescaling image from -1 to 1 to 0 to 1
    imgs = imgs * 0.5 + 0.5

    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
        for j in range(cols):
            axs[i,j].imshow(imgs[idx].reshape(512,512,3))
            axs[i,j].axis("off")
            idx += 1
    fig.savefig("/Users/t.kan/Desktop/CNN/gan_images/%d.png" % epoch)
    plt.close()

In [ ]:
for epoch in range(epochs):

  ## train discriminator ##

  # get random batches of real images
    idx = np.random.randint(0, 8, batch_size)
    real_imgs = im_array[idx]

  # generate fake images
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)

  # use train_on_batch to train discriminator
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = (d_loss_real + d_loss_fake) / 2
    d_acc = (d_acc_real + d_acc_fake) / 2

  ## train generator ##

  # by calling combined model and optimize for the freezed discriminator values to ones
  # we can optimize generator to generate real like images
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)

    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch % 10 == 0:
        print(f"epoch: {epoch+1} / {epochs}, d_loss: {d_loss:.2f}, d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")

    if epoch % sample_period == 0:
        sample_images(epoch)
        gen_weights = generator.get_weights()
        dis_weights = discriminator.get_weights()
        np.save(os.path.join("/Users/t.kan/Desktop/CNN/model_dir", 'dcgan_generator_weights'), gen_weights)
        np.save(os.path.join("/Users/t.kan/Desktop/CNN/model_dir", 'dcgan_discriminator_weights'), dis_weights)
        

generator.save(os.path.join("/Users/t.kan/Desktop/CNN/model_dir", 'dcgan_generator.h5'))
discriminator.save(os.path.join("/Users/t.kan/Desktop/CNN/model_dir", 'dcgan_discriminator.h5'))

4/4 [==============================] - 106s 21s/step
